In [1]:
from astropy import units as u
from astropy.coordinates import Angle
from astropy.coordinates import SkyCoord
from astropy.io import ascii
from astropy.table import setdiff, Table, unique, vstack
import numpy as np
%run Crossmatch.ipynb
%run Utility.ipynb

In [2]:
## NEW CELL WITH SEQUENTIAL AVERAGING ##
%run Crossmatch.ipynb
total_duplicates = Table()

# Path for aperture photometry based PSF-modelled catalogs
path = '/home/yobd/Documents/Photometry_Output/Zero_Point_and_Color_Term_Aper_Model_Individually/'

averaged = ascii.read('/home/yobd/Documents/Photometry_Output/Zero_Point_and_Color_Term_Aper_Model_Individually/Chihiro.dat')

# Encodes the path of the dataset to be used by other OS methods

folder = ['Clarisse', 'Fio', 'Kiki', 'Nausicaa', 'Ponyo', 'San', 'Satsuki', 'Sheeta', 'Sophie']

# Cycles through the folder where the catalogs are stored - They will be cross-matched with each other in order to find all the duplicates in our images

for item in folder:
    
    # The name of the current catalog
    
    item_name = item
    
    item = path + item + '.dat' # To comply with the func() function parameter
    
    # Opens the current catalog as an astropy.Table object
    
    cat = ascii.read(item)
    
    # The astropy.Table object containing the cross-matches between the current catalog and the second catalog being iterated
    
    temp_duplicates = cross_match(averaged, 'ra', 'dec', cat, 'ra', 'dec', 3, sep = True)
    
    # Only perform the following actions if there are cross-matches, otherwise it will give an error (i.e., when cross-matching catalogs at opposite ends of the instrument)
    
    if len(temp_duplicates) != 0:
        
        # Resets the columns of the current catalog back to their original names
        
        temp_duplicates = reset_main_catalog_columns(averaged, temp_duplicates)
        
        temp_duplicates.sort('d2d')
        
        temp_duplicates = unique(temp_duplicates, keys = ['ra_2', 'dec_2'], keep = 'first')
        
        singles = setdiff(averaged, temp_duplicates, keys = ['ra', 'dec'])
        
        cat['ra_2'] = cat['ra']
        
        cat['dec_2'] = cat['dec']
        
        singles_cat = setdiff(cat, temp_duplicates, keys = ['ra_2', 'dec_2'])
        
        remove_new_columns(averaged, singles_cat)
        
        # Adds the duplicates from the current catalog - second catalog pair onto the astropy.Table containing all the duplicates across all detectors
        
        total_duplicates = vstack([total_duplicates, temp_duplicates])
        
        temp_duplicates['ra'] = (temp_duplicates['ra'] + temp_duplicates['ra_2']) / 2
        
        temp_duplicates['dec'] = (temp_duplicates['dec'] + temp_duplicates['dec_2']) / 2
        
        temp_duplicates['I_MAG_SUBARU'] = (temp_duplicates['I_MAG_SUBARU'] + temp_duplicates['I_MAG_SUBARU_2']) / 2
        
        temp_duplicates['I_MAG_SUBARU_ERR'] = np.sqrt(0.25 * temp_duplicates['I_MAG_SUBARU_ERR'] * temp_duplicates['I_MAG_SUBARU_ERR'] + 0.25 * temp_duplicates['I_MAG_SUBARU_ERR_2'] * temp_duplicates['I_MAG_SUBARU_ERR_2'])
        
        temp_duplicates['I_FLUX_SUBARU'] = (temp_duplicates['I_FLUX_SUBARU'] + temp_duplicates['I_FLUX_SUBARU_2']) / 2
        
        temp_duplicates['I_FLUX_SUBARU_ERR'] = np.sqrt(0.25 * temp_duplicates['I_FLUX_SUBARU_ERR'] * temp_duplicates['I_FLUX_SUBARU_ERR'] + 0.25 * temp_duplicates['I_FLUX_SUBARU_ERR_2'] * temp_duplicates['I_FLUX_SUBARU_ERR_2'])
        
        temp_duplicates['I_FLUX_RADIUS_SUBARU'] = (temp_duplicates['I_FLUX_RADIUS_SUBARU'] + temp_duplicates['I_FLUX_RADIUS_SUBARU_2']) / 2
        
        temp_duplicates['ELONGATION'] = (temp_duplicates['ELONGATION'] + temp_duplicates['ELONGATION_2']) / 2
        
        temp_duplicates['ELLIPTICITY'] = (temp_duplicates['ELLIPTICITY'] + temp_duplicates['ELLIPTICITY_2']) / 2
        
        temp_duplicates['I_SNR_WIN_SUBARU'] = (temp_duplicates['I_SNR_WIN_SUBARU'] + temp_duplicates['I_SNR_WIN_SUBARU_2']) / 2
        
        #temp_duplicates['FLAGS'] = temp_duplicates['FLAGS'] + temp_duplicates['FLAGS_2']
        
        #temp_duplicates['FLAGS_MODEL'] = temp_duplicates['FLAGS_MODEL'] + temp_duplicates['FLAGS_MODEL_2']
        
        #temp_duplicates['I_FLUX_APER_SUBARU'] = (temp_duplicates['I_FLUX_APER_SUBARU'] + temp_duplicates['I_FLUX_APER_SUBARU_2']) / 2
        
        #temp_duplicates['I_FLUX_APER_SUBARU_ERR'] = np.sqrt(0.25 * temp_duplicates['I_FLUX_APER_SUBARU_ERR'] * temp_duplicates['I_FLUX_APER_SUBARU_ERR'] + 0.25 * temp_duplicates['I_FLUX_APER_SUBARU_ERR_2'] * temp_duplicates['I_FLUX_APER_SUBARU_ERR_2'])
        
        # Removes the columns from the cross-matched second catalog
        
        remove_new_columns(averaged, temp_duplicates)
        
        averaged = vstack([singles, temp_duplicates])
        
        averaged = vstack([averaged, singles_cat])


# Path for aperture photometry based PSF-modelled catalogs
ascii.write(averaged, '/home/yobd/Documents/Photometry_Output/Full_Catalog_ZP_CT_Aper_PSF.dat', format = 'csv', overwrite = True)



Clarisse
Fio
Kiki
Nausicaa
Ponyo
San
Satsuki
Sheeta
Sophie
        ra                 dec         ...     ksAperMag3Err     DETECTOR
------------------ ------------------- ... --------------------- --------
285.08119619898883  -36.92538333879947 ...    0.1610533744096756 Nausicaa
285.08133561427456  -36.92997296952866 ...   0.13872365653514862 Nausicaa
285.08152146155703 -36.946266021430326 ...   0.17129318416118622 Nausicaa
285.08164700778866   -36.9128840942102 ...    0.1999858170747757 Nausicaa
285.08191778038326  -36.90810164049388 ...          -999999488.0 Nausicaa
285.08208616963077  -36.87260750280702 ...   0.04557235911488533 Nausicaa
 285.0826123766585  -36.84959781376215 ...   0.04723486676812172 Nausicaa
 285.0826325972246  -36.92841693627763 ...          -999999488.0 Nausicaa
 285.0830768103179  -36.83753529766357 ...   0.04441603273153305 Nausicaa
285.08310339170777  -36.87034207038855 ...          -999999488.0 Nausicaa
               ...                 ... ...           

In [3]:
## The duplicates code for the H-Alpha catalog using last cell's method

%run Crossmatch.ipynb
total_duplicates = Table()

# The path where the dataset is located

path = '/home/yobd/Documents/AstroAliSciObj/N-A-L656/MINAREA_4_THRESH_2_dot_7_SNR_20_SATUR_LEVEL_B4_PSFEX_30000_SMP_VAR_0_dot_2_SEEING_6_dot_7_7_MAG_PSF/Catalogs_Finished/'

#path = '/home/yobd/Documents/Photometry_Output/Zero_Point_and_Color_Term/'

averaged = ascii.read('/home/yobd/Documents/AstroAliSciObj/N-A-L656/MINAREA_4_THRESH_2_dot_7_SNR_20_SATUR_LEVEL_B4_PSFEX_30000_SMP_VAR_0_dot_2_SEEING_6_dot_7_7_MAG_PSF/Catalogs_Finished/N-A-L656_chihiro_stacked_tmp.dat')

# Encodes the path of the dataset to be used by other OS methods

folder = ['N-A-L656_clarisse_stacked_tmp', 'N-A-L656_fio_stacked_tmp', 'N-A-L656_kiki_stacked_tmp', 'N-A-L656_nausicaa_stacked_tmp', 'N-A-L656_ponyo_stacked_tmp', 'N-A-L656_san_stacked_tmp', 'N-A-L656_satsuki_stacked_tmp', 'N-A-L656_sheeta_stacked_tmp', 'N-A-L656_sophie_stacked_tmp']

# Cycles through the folder where the catalogs are stored - They will be cross-matched with each other in order to find all the duplicates in our images

for item in folder:
    
    # The name of the current catalog
    
    item_name = item
    
    print(item_name)
    
    item = path + item + '.dat' # To comply with the func() function parameter
    
    # Opens the current catalog as an astropy.Table object
    
    cat = ascii.read(item)
    
    # The astropy.Table object containing the cross-matches between the current catalog and the second catalog being iterated
    
    temp_duplicates = cross_match(averaged, 'ra', 'dec', cat, 'ra', 'dec', 3, sep = True)
    
    # Only perform the following actions if there are cross-matches, otherwise it will give an error (i.e., when cross-matching catalogs at opposite ends of the instrument)
    
    if len(temp_duplicates) != 0:
        
        # Resets the columns of the current catalog back to their original names
        
        temp_duplicates = reset_main_catalog_columns(averaged, temp_duplicates)
        
        temp_duplicates.sort('d2d')
        
        temp_duplicates = unique(temp_duplicates, keys = ['ra_2', 'dec_2'], keep = 'first')
        
        singles = setdiff(averaged, temp_duplicates, keys = ['ra', 'dec'])
        
        cat['ra_2'] = cat['ra']
        
        cat['dec_2'] = cat['dec']
        
        singles_cat = setdiff(cat, temp_duplicates, keys = ['ra_2', 'dec_2'])
        
        remove_new_columns(averaged, singles_cat)
        
        # Adds the duplicates from the current catalog - second catalog pair onto the astropy.Table containing all the duplicates across all detectors
        
        total_duplicates = vstack([total_duplicates, temp_duplicates])
        
        temp_duplicates['ra'] = (temp_duplicates['ra'] + temp_duplicates['ra_2']) / 2
        
        temp_duplicates['dec'] = (temp_duplicates['dec'] + temp_duplicates['dec_2']) / 2
        
        temp_duplicates['FLUX_PSF'] = (temp_duplicates['FLUX_PSF'] + temp_duplicates['FLUX_PSF_2']) / 2
        
        temp_duplicates['FLUXERR_PSF'] = np.sqrt(0.25 * temp_duplicates['FLUXERR_PSF'] * temp_duplicates['FLUXERR_PSF'] + 0.25 * temp_duplicates['FLUXERR_PSF_2'] * temp_duplicates['FLUXERR_PSF_2'])
        
        temp_duplicates['FLUX_RADIUS'] = (temp_duplicates['FLUX_RADIUS'] + temp_duplicates['FLUX_RADIUS_2']) / 2
        
        temp_duplicates['ELONGATION'] = (temp_duplicates['ELONGATION'] + temp_duplicates['ELONGATION_2']) / 2
        
        temp_duplicates['ELLIPTICITY'] = (temp_duplicates['ELLIPTICITY'] + temp_duplicates['ELLIPTICITY_2']) / 2
        
        temp_duplicates['SNR_WIN'] = (temp_duplicates['SNR_WIN'] + temp_duplicates['SNR_WIN_2']) / 2
        
        temp_duplicates['FLAGS'] = temp_duplicates['FLAGS'] + temp_duplicates['FLAGS_2']
        
        temp_duplicates['FLAGS_MODEL'] = temp_duplicates['FLAGS_MODEL'] + temp_duplicates['FLAGS_MODEL_2']
        
        # Removes the columns from the cross-matched second catalog
        remove_new_columns(averaged, temp_duplicates)
        
        averaged = vstack([singles, temp_duplicates])
        
        averaged = vstack([averaged, singles_cat])

print("Length of AVERAGED before flag cut:", len(averaged))
averaged.remove_rows([averaged['FLAGS'] != 0])
print("Length of AVERAGED after flag cut:", len(averaged))

ascii.write(averaged, '/home/yobd/Documents/Photometry_Output/N-A-L656_Full_Catalog_Aper_PSF.dat', format = 'csv', overwrite = True)

print(averaged)

N-A-L656_clarisse_stacked_tmp
N-A-L656_fio_stacked_tmp
N-A-L656_kiki_stacked_tmp
N-A-L656_nausicaa_stacked_tmp
N-A-L656_ponyo_stacked_tmp
N-A-L656_san_stacked_tmp
N-A-L656_satsuki_stacked_tmp
N-A-L656_sheeta_stacked_tmp
N-A-L656_sophie_stacked_tmp
Length of AVERAGED before flag cut: 26754
Length of AVERAGED after flag cut: 22969
NUMBER  X_IMAGE   Y_IMAGE  ... SNR_WIN         ra                 dec        
------ --------- --------- ... ------- ------------------ -------------------
    38 2834.3274   404.391 ...   16.46 285.08145135963935 -36.946252566582736
   167 2834.3848 1090.4749 ...   31.41 285.08181148555633   -36.9081215652054
   240 2828.0273 1268.0155 ...    13.3 285.08235218599384  -36.89827641052004
    90 2820.5254  724.6572 ...   29.08 285.08254652093296  -36.92843782185453
   667 2829.4248 1887.1907 ...   25.88  285.0826881832103 -36.864010804457635
   551 2824.3071  1772.493 ...    68.3 285.08294945923797  -36.87034964225969
   587 2824.1995  1811.001 ...   33.67 285.08

/home/yobd/.local/lib/python3.7/site-packages/astropy/table/table.py:2020: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  keep_mask[row_specifier] = False


In [2]:
## CELL FOR CATALOG BEFORE PHOTOMETRY USING SEQUENTIAL AVERAGING ##

%run Crossmatch.ipynb

total_duplicates = Table()

# The path where the dataset is located

path = '/home/yobd/Documents/AstroAliSciObj/Catalogs_Finished_4_1_dot_8_2_5/W-S-I+/'

averaged = ascii.read('/home/yobd/Documents/AstroAliSciObj/Catalogs_Finished_4_1_dot_8_2_5/W-S-I+/W-S-I+_chihiro_stacked_tmp.cat')

averaged.remove_rows([averaged['FLAGS'] != 0])

averaged.remove_rows([averaged['FLUX_AUTO'] == 0])

averaged.remove_rows([averaged['ELLIPTICITY'] >= 0.5])

# Encodes the path of the dataset to be used by other OS methods

folder = ['W-S-I+_clarisse_stacked_tmp', 'W-S-I+_fio_stacked_tmp', 'W-S-I+_kiki_stacked_tmp', 'W-S-I+_nausicaa_stacked_tmp', 'W-S-I+_ponyo_stacked_tmp', 'W-S-I+_san_stacked_tmp', 'W-S-I+_satsuki_stacked_tmp', 'W-S-I+_sheeta_stacked_tmp', 'W-S-I+_sophie_stacked_tmp']

# Cycles through the folder where the catalogs are stored - They will be cross-matched with each other in order to find all the duplicates in our images

for item in folder:
    
    # The name of the current catalog
    
    item_name = item
    
    print(item_name)
    
    item = path + item + '.cat' # To comply with the func() function parameter
    
    # Opens the current catalog as an astropy.Table object
    
    cat = ascii.read(item)
    
    cat.remove_rows([cat['FLAGS'] != 0])
    
    cat.remove_rows([cat['FLUX_AUTO'] == 0])
    
    cat.remove_rows([cat['ELLIPTICITY'] >= 0.5])
    
    # The astropy.Table object containing the cross-matches between the current catalog and the second catalog being iterated
    
    temp_duplicates = cross_match(averaged, 'ra', 'dec', cat, 'ra', 'dec', 3, sep = True)
    
    # Only perform the following actions if there are cross-matches, otherwise it will give an error (i.e., when cross-matching catalogs at opposite ends of the instrument)
    
    if len(temp_duplicates) != 0:
        
        # Resets the columns of the current catalog back to their original names
        
        temp_duplicates = reset_main_catalog_columns(averaged, temp_duplicates)
        
        temp_duplicates.sort('d2d')
        
        temp_duplicates = unique(temp_duplicates, keys = ['ra_2', 'dec_2'], keep = 'first')
        
        singles = setdiff(averaged, temp_duplicates, keys = ['ra', 'dec'])
        
        cat['ra_2'] = cat['ra']
        
        cat['dec_2'] = cat['dec']
        
        singles_cat = setdiff(cat, temp_duplicates, keys = ['ra_2', 'dec_2'])
        
        remove_new_columns(averaged, singles_cat)
        
        # Adds the duplicates from the current catalog - second catalog pair onto the astropy.Table containing all the duplicates across all detectors
        
        total_duplicates = vstack([total_duplicates, temp_duplicates])
        
        temp_duplicates['ra'] = (temp_duplicates['ra'] + temp_duplicates['ra_2']) / 2
        
        temp_duplicates['dec'] = (temp_duplicates['dec'] + temp_duplicates['dec_2']) / 2
        
        temp_duplicates['X_IMAGE'] = (temp_duplicates['X_IMAGE'] + temp_duplicates['X_IMAGE_2']) / 2
        
        temp_duplicates['Y_IMAGE'] = (temp_duplicates['Y_IMAGE'] + temp_duplicates['Y_IMAGE_2']) / 2
        
        temp_duplicates['ALPHA_J2000'] = (temp_duplicates['ALPHA_J2000'] + temp_duplicates['ALPHA_J2000_2']) / 2
        
        temp_duplicates['DELTA_J2000'] = (temp_duplicates['DELTA_J2000'] + temp_duplicates['DELTA_J2000_2']) / 2
        
        temp_duplicates['FLUX_AUTO'] = (temp_duplicates['FLUX_AUTO'] + temp_duplicates['FLUX_AUTO_2']) / 2
        
        temp_duplicates['FLUXERR_AUTO'] = np.sqrt(0.25 * temp_duplicates['FLUXERR_AUTO'] * temp_duplicates['FLUXERR_AUTO'] + 0.25 * temp_duplicates['FLUXERR_AUTO_2'] * temp_duplicates['FLUXERR_AUTO_2'])
        
        temp_duplicates['FLUX_RADIUS'] = (temp_duplicates['FLUX_RADIUS'] + temp_duplicates['FLUX_RADIUS_2']) / 2
        
        temp_duplicates['ELONGATION'] = (temp_duplicates['ELONGATION'] + temp_duplicates['ELONGATION_2']) / 2
        
        temp_duplicates['ELLIPTICITY'] = (temp_duplicates['ELLIPTICITY'] + temp_duplicates['ELLIPTICITY_2']) / 2
        
        temp_duplicates['SNR_WIN'] = (temp_duplicates['SNR_WIN'] + temp_duplicates['SNR_WIN_2']) / 2
        
        temp_duplicates['FLUX_APER'] = (temp_duplicates['FLUX_APER'] + temp_duplicates['FLUX_APER_2']) / 2
        
        temp_duplicates['FLUXERR_APER'] = np.sqrt(0.25 * temp_duplicates['FLUXERR_APER'] * temp_duplicates['FLUXERR_APER'] + 0.25 * temp_duplicates['FLUXERR_APER_2'] * temp_duplicates['FLUXERR_APER_2'])
        
        # Removes the columns from the cross-matched second catalog
        
        remove_new_columns(averaged, temp_duplicates)
        
        averaged = vstack([singles, temp_duplicates])
        
        averaged = vstack([averaged, singles_cat])

ascii.write(averaged, '/home/yobd/Documents/AstroAliSciObj/Catalogs_Finished_4_1_dot_8_2_5/W-S-I+_Full_Catalog_no_duplicates.dat', format = 'csv', overwrite = True)

print(averaged)

/home/yobd/.local/lib/python3.7/site-packages/astropy/table/table.py:2020: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  keep_mask[row_specifier] = False


W-S-I+_clarisse_stacked_tmp
W-S-I+_fio_stacked_tmp
W-S-I+_kiki_stacked_tmp


/home/yobd/.local/lib/python3.7/site-packages/astropy/table/table.py:2020: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  keep_mask[row_specifier] = False
/home/yobd/.local/lib/python3.7/site-packages/astropy/table/table.py:2020: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  keep_mask[row_specifier] = False


W-S-I+_nausicaa_stacked_tmp


/home/yobd/.local/lib/python3.7/site-packages/astropy/table/table.py:2020: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  keep_mask[row_specifier] = False
/home/yobd/.local/lib/python3.7/site-packages/astropy/table/table.py:2020: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  keep_mask[row_specifier] = False


...

W-S-I+_satsuki_stacked_tmp
W-S-I+_sheeta_stacked_tmp


/home/yobd/.local/lib/python3.7/site-packages/astropy/table/table.py:2020: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  keep_mask[row_specifier] = False


W-S-I+_sophie_stacked_tmp


/home/yobd/.local/lib/python3.7/site-packages/astropy/table/table.py:2020: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  keep_mask[row_specifier] = False


NUMBER  X_IMAGE   Y_IMAGE  ...         ra                 dec        
------ --------- --------- ... ------------------ -------------------
   149 2833.5979  529.8074 ...  285.0809916661293 -36.938937629844744
   229 2832.6501  773.8334 ... 285.08120009837006 -36.925378182945174
   306 2833.4841  960.5045 ... 285.08126073184445  -36.91501670009131
   196 2829.9634  691.1179 ...   285.081332207662  -36.92997269382326
   472 2833.3518 1231.3008 ...   285.081450186502  -36.90000286998675
   104 2824.8125  398.0334 ... 285.08151390947955  -36.94626764930463
   311 2827.8784  998.5699 ... 285.08166507228805 -36.912905094181866
   252 2825.5286  809.5094 ... 285.08170463018445 -36.923397534577255
   358 2824.9929 1085.2549 ...  285.0819174153343 -36.908097620533304
   637 2826.5627 1405.7424 ...  285.0820325499118 -36.890341596865966
   ...       ...       ... ...                ...                 ...
  4700   26.1663 2109.6592 ...  285.2739416833508 -37.088459422976285
  3778   27.4028  16